### 강아지와 고양이 사진 분류 모델의 성능 개선하기 목차

* [Chapter 1 잔차 블록(residual block)](#chapter1)
* [Chapter 2 ReNet 모델 만들기](#chapter2)
* [Chapter 3 합성곱층 만들기](#chapter3)
   * [Section 3.1 입력과 출력사이의 관계](#section_3_1)
   * [Section 3.2 풀링층과 밀집층](#section_3_2)

### Chapter 1 잔차 블록(residual block) <a class="anchor" id="chapter1"></a>
1. 딥러닝 모델의 훈련과정
   - 경사 하강법으로 신경망 훈련 --> 손실 함수를 통해 모델의 출력과 타깃 사의의 오차 계산
     --> 모델의 끝에서부터 앞으로 누적하여 계산하는 역전파
   - 역전파: 출력 값과 실제 값의 차이를 확인하는 과정에서 오차 발새의 원인을 찾고, 신경망의 가중치를 조정하는 역활을 한다.

2. 딥러닝 모델은 신경망이 깊어질수록 이 가중치를 변경해 오차을 줄이는 그레디언트가 점점 작아진다.
   - 입력 부분에 가까운 가중치가 잘 변경하지 않는 문제가 발생한다.

3. 잔차 블록(residual block)은 입력을 출력에 직접 연결하는 스킵 연결을 추가해 그레디언트 소실을 완화한다.
   - 신경망의 층이 깊어지더라도 그레디언트가 잘 전파되어 신경망의 모든 층이 잘 훈련되는 효과를 낸다.

   ![잔차 블록](image/02-03-sidualBlock.png)    

4. ReNet50, ReNet101, ReNet152에서 사용하는 잔차 블록은 ReNet18, ReNet345의 잔차 블록과 좀 다르다.
   - 합성곱 층 다음에 배치 정규화가 추가된다.
   - 모델의 연산량을 줄이기 위해 3개의 합성곱층으로 구성된 병목 블록(bottleneck block)을 구성한다.
      - 첫 번재 합성곱층: 입력의 공간 방향의 크기를 출인다.
      - 두 번째 합성곱층: 입력 채널의 크기를 유지한 채 특징을 추출한다.
      - 세 번재 합성곱층: 채널의 수를 확장한다.

    ![잔차 블록2](image/02-03-sidualBlock2.png)    



### Chapter 2 ReNet 모델 만들기 <a class="anchor" id="chapter2"></a>
1. ReNet 모델은 잔차 모듈이라는 구조를 통해 더욱 깊은 신경망에서도 모델의 학습 성능을 향상시킨다.

2. 잔차 모듈은 스킵연결을 통해 입력 데이터를 직접 다음 층으로 전달한다.
   - 신경망이 깊어지더라도 학습이 가능하도록 돕는다.
   - 잔차 블록과 그 잔차 블록이 모인 잔차 스텍으로 구성된다.

3. 배치 정규화
   - 잔차 블록 내에서 학습의 속도를 높이고 모델의 안전성을 개선하기위해 사용한다.
   - 신경망 입력은 보통 표준화를 통해 평균이 0, 분산이 1이 되도록 정규화된다.
   - 입력 데이터가 신경망의 여러층을 통과하면서 이런 정규화가 깨질 수 있다.
   - 배치 정규화 층은 배치 단위로 다시 정규화 함으로써 훈련의 속도와 성능을 높인다.
   - 샘플의 평균을 계산한다.
      - m: 배치에 있는 샘플 개수, B: 배치에 대한 평균임을 표시

         ![Avg](image/02-03-avg3.png) 

   - 분산을 계산한다.
      - B: 배치에 대한 분산임을 표시

         ![분산](image/02-03-var.png) 

   - 정규화를 진행한다.
      - 입력에서 평균을 빼고 표준 편차로 나눈다.
      - 분모가 0이되지 않도록 아주 작은 입실론(∈: 1*10^-5, 기본값 0.001)을 더한다.

          ![표준화](image/02-03-stand.png) 

   - 정규화가 진행된 값에 감와 베타를 더해 최종 출력을 만든다.
      - 정규화된 값은 평균이 0, 표준 편차가 1이므로 시그모이드 곡선의 직선에 대부분 위치하게되어 효과가 줄어든다.
      - 감마와 베타를 더하여 평균값의 범위를 변경한다.
      - 감마와 베타값도 역전파를 통해 학습된다.

         ![표준화2](image/02-03-stand2.png)
   
         
4. ReNet 전체구조   
   ![ReNet](image/02-03-ReNet.png)    

In [ ]:
import keras 
from keras import layers

# 입력 크기 정의
inputs = layers.Input(shape=(224, 224, 3))

# padding=3 : 이미지 위/아래 각각 3픽셀, 좌/우 각각 3픽셀씩 0으로 채움
# 출력 크기 : (224 + 3*2, 224 + 3*2) = (230, 230)
x = layers.ZeroPadding2D(padding=3)(inputs)

#7*7 필터 64개, 스트라이드 2 합성곱 층에 입력 데이터 x 전달
# 출력 크기 : ( (230 - 7) / 2 + 1, (230 - 7) / 2 + 1 ) = (112, 112)
x = layers.Conv2D(64, kernel_size=7, strides=2)(x)

# 배치 정규화 층에 입력 데이터 x 전달
#   - epsilon : 분모가 0이 되는 것을 방지하기 위한 아주 작은 값을 분산에 더한다.
x = layers.BatchNormalization(epsilon=1e-5)(x)
x = layers.Activation('relu')(x)